## Import necessary libraries

In [2]:
import urllib
import json
import numpy as np 
import pandas as pd
import os
import zipfile
import cv2
from matplotlib import pyplot as plt
import shutil 
from tqdm import tqdm
import torch
import torchvision
import time
import copy
import random
from torchvision import transforms, models
from foolbox.attacks import L2ProjectedGradientDescentAttack
from foolbox import PyTorchModel

# in case CUDA is not available, shutdown notebook and reload GPU in terminal:
#     sudo rmmod nvidia_uvm
#     sudo modprobe nvidia_uvm

## Some constants

In [3]:
IMG_SIZE = 64
SEED_VALUE = 666
BATCH_SIZE = 64

## Set all seeds

In [4]:
def InitSeeds(seed_value):
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    os.environ["PYTHONHASHSEED"] = str(seed_value)

InitSeeds(SEED_VALUE)

## Image transforms

In [5]:
test_transforms = transforms.Compose([
    #transforms.RandomResizedCrop(IMG_SIZE, scale=(1.0, 1.0), ratio=(1.0, 1.1)),
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

## Load trained network

In [10]:
model = models.resnet34(pretrained=True)
model.fc = torch.nn.Linear(model.fc.in_features, 10)
model.load_state_dict(torch.load("base model.pth"))
model.eval()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Network will be trained on {device} device")
model = model.to(device)
optimizer = torch.optim.Adam(model.parameters(), amsgrad=True, lr=0.01)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.75)

Network will be trained on cuda:0 device


## Test network on original test images

In [12]:
test_dir = 'AHE test/Dataset_test_64/'   
test_dataset = torchvision.datasets.ImageFolder('img/'+test_dir, test_transforms)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=1, shuffle=False, num_workers=8)

phase = 'test'
dataloader = test_dataloader

running_loss = 0. 
running_acc = 0.
loss_func = torch.nn.CrossEntropyLoss()

for inputs, labels in tqdm(dataloader):
    inputs = inputs.to(device) # Тензор с изображениями переводим на GPU 
    labels = labels.to(device) # Тензор с лейблами переводим на GPU 
 
    optimizer.zero_grad() # Обнуляем градиент,чтобы он не накапливался 
 
    with torch.set_grad_enabled(phase == 'train'): #Если фаза train то активируем все градиенты (те которые не заморожены) (очистить историю loss)
        preds = model(inputs) # Считаем предикты, input передаем в модель
        loss_value = loss_func(preds, labels) #Посчитали  Loss
        preds_class = preds.argmax(dim=1) # Получаем класс,берем .argmax(dim=1) нейрон с максимальной активацией
    # Статистика
    running_loss += loss_value.item() #считаем Loss
    running_acc += (preds_class == labels.data).float().mean().data.cpu().numpy()  #считаем accuracy
 
epoch_loss = running_loss / len(dataloader)  # Loss'ы делим на кол-во бачей в эпохе 
epoch_acc = running_acc / len(dataloader) #считаем Loss на кол-во бачей в эпохе
 
print("{} Loss: {:.4f} Acc: {:.4f}".format(phase, epoch_loss, epoch_acc), end="")

100%|██████████| 1404/1404 [00:31<00:00, 44.07it/s]

test Loss: 0.8746 Acc: 0.7764

## Test network on attacked images

In [19]:
# Generate attacked images
class ImageFolderWithPaths(torchvision.datasets.ImageFolder):
    def __getitem__(self, index):
        original_tuple = super(ImageFolderWithPaths, self).__getitem__(index)
        path = self.imgs[index][0]
        tuple_with_path = (original_tuple + (path,))
        return tuple_with_path
    
def UnNormalize(norm_img: torch.Tensor) -> torch.Tensor:
    mean = torch.tensor([0.485, 0.456, 0.406]).reshape(1,3,1,1).to(device)
    std = torch.tensor([0.229, 0.224, 0.225]).reshape(1,3,1,1).to(device)
    return norm_img*std + mean

model_for_attacks = copy.deepcopy(model)
for param in model_for_attacks.parameters():
    param.requires_grad = True

test_dataset = ImageFolderWithPaths('img/'+test_dir, test_transforms)

test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=1, shuffle=False, num_workers=8)

fmodel = PyTorchModel(model, bounds=(0, 1))

L2PGDattack = L2ProjectedGradientDescentAttack(steps = 50)

success_count = 0

for inputs, labels, paths in tqdm(test_dataloader):
    #print(inputs, labels, paths)
    inputs = inputs.to(device) # Тензор с изображениями переводим на GPU 
    labels = labels.to(device) # Тензор с лейблами переводим на GPU 
    _, bacth_adv, success = L2PGDattack(fmodel, inputs, labels, epsilons = [128/255])
    img_adv = UnNormalize(bacth_adv[0])
    #print(int(success[0][0]))
    success_count += int(success[0][0])
    #img_adv = bacth_adv[0]
    #print("Generated adversarial image:\n", img_adv[0])
    img_class = paths[0].split("/")[-2]
    img_name = paths[0].split("/")[-1].rsplit(".", maxsplit = 1)[0] +\
               "_attacked" + "." +\
               paths[0].split("/")[-1].rsplit(".", maxsplit = 1)[1]
    img_path = os.path.join("img", "test attacked", img_class, img_name)
    os.makedirs(os.path.join('img', "test attacked", img_class), exist_ok = True)
    torchvision.utils.save_image(img_adv[0], img_path)
print(f"Accuracy on attacked images: {1 - success_count/len(test_dataset)}")

100%|██████████| 1404/1404 [51:52<00:00,  2.22s/it]

Accuracy on attacked images: 0.774928774928775


In [21]:
# Test on attacked images
test_dir = 'test attacked'
    
test_dataset = torchvision.datasets.ImageFolder('img/'+test_dir, test_transforms)

test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=4, shuffle=False, num_workers=8)

phase = 'test'
dataloader = test_dataloader
model.eval()
 
running_loss = 0. 
running_acc = 0.
loss_func = torch.nn.CrossEntropyLoss()
 
# Итерируемся по dataloader
for inputs, labels in tqdm(dataloader):
    inputs = inputs.to(device) # Тензор с изображениями переводим на GPU 
    labels = labels.to(device) # Тензор с лейблами переводим на GPU 
 
    optimizer.zero_grad() # Обнуляем градиент,чтобы он не накапливался 
 
    with torch.set_grad_enabled(phase == 'train'): #Если фаза train то активируем все градиенты (те которые не заморожены) (очистить историю loss)
        preds = model(inputs) # Считаем предикты, input передаем в модель
        loss_value = loss_func(preds, labels) #Посчитали  Loss
        preds_class = preds.argmax(dim=1) # Получаем класс,берем .argmax(dim=1) нейрон с максимальной активацией
    # Статистика
    running_loss += loss_value.item() #считаем Loss
    running_acc += (preds_class == labels.data).float().mean().data.cpu().numpy()  #считаем accuracy
 
epoch_loss = running_loss / len(dataloader)  # Loss'ы делим на кол-во бачей в эпохе 
epoch_acc = running_acc / len(dataloader) #считаем Loss на кол-во бачей в эпохе
 
print("{} Loss: {:.4f} Acc: {:.4f}".format(phase, epoch_loss, epoch_acc), end="")

100%|██████████| 702/702 [00:14<00:00, 49.86it/s]

test Loss: 0.9228 Acc: 0.7610